# Assignment 6: Reduction of dimensionality and recognition

### Exercise 1: Direct PCA method

#### a) Analytically compute eigenvectors and eigenvalues given four points.

#### b) Calculate and visualize PCA for 2D data from `points.txt`.